In [448]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import geopandas as gpd
from census import Census

In [449]:
path = '/Users/nick0o0o0/Library/Mobile Documents/com~apple~CloudDocs/gxyfile/third_year/ECO353/Assignment/project1/data/'

NBA_attend = pd.read_excel(path + 'NBA Attend Index.xls')
NBA_revenue = pd.read_csv(path+ 'Finance.csv')

In [450]:
NBA_revenue['Team'] = NBA_revenue['Team'].str.title()
NBA_revenue['Year2'] = NBA_revenue['Year'].str[:4]

In [451]:
team_name_dict = {
    'Atlanta': 'Atlanta Hawks',
    'Boston': 'Boston Celtics',
    'Brooklyn Nets': 'Brooklyn Nets',
    'Charlotte': 'Charlotte Hornets',
    'Charlotte Bobcats': 'Charlotte Hornets',
    'Chicago': 'Chicago Bulls',
    'Cleveland': 'Cleveland Cavaliers',
    'Dallas': 'Dallas Mavericks',
    'Dallas Mavericks/': 'Dallas Mavericks',
    'Denver': 'Denver Nuggets',
    'Detroit': 'Detroit Pistons',
    'Golden State': 'Golden State Warriors',
    'Houston': 'Houston Rockets',
    'Indiana': 'Indiana Pacers',
    'Los Angeles Clippers': 'Los Angeles Clippers',
    'Los Angeles Lakers': 'Los Angeles Lakers',
    'Memphis': 'Memphis Grizzlies',
    'Miami': 'Miami Heat',
    'Milwaukee': 'Milwaukee Bucks',
    'Minnesota': 'Minnesota Timberwolves',
    'New Jersey': 'Brooklyn Nets',
    'New Jersey Nets': 'Brooklyn Nets',
    'New Orleans': 'New Orleans Pelicans',
    'New Orleans Hornets': 'New Orleans Pelicans',
    'New Orleans Pelicans': 'New Orleans Pelicans',
    'New York': 'New York Knicks',
    'Oklahoma City': 'Oklahoma City Thunder',
    'Orlando': 'Orlando Magic',
    'Philadelphia': 'Philadelphia 76ers',
    'Philadelphia 76Ers': 'Philadelphia 76ers',
    'Phoenix': 'Phoenix Suns',
    'Portland': 'Portland Trail Blazers',
    'Sacramento': 'Sacramento Kings',
    'San Antonio': 'San Antonio Spurs',
    'Seattle Supersonics': 'Oklahoma City Thunder',
    'Toronto': 'Toronto Raptors',
    'Utah': 'Utah Jazz',
    'Vancouver Grizzlies': 'Memphis Grizzlies',
    'Washington': 'Washington Wizards',
    'Washington Wizards': 'Washington Wizards'
}

# apply the mapping to the 'Team' column of your dataframe
NBA_revenue['Team'] = NBA_revenue['Team'].apply(lambda x: team_name_dict.get(x, x))

In [452]:
set(NBA_revenue['Team'])

{'Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'League Average',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards'}

In [453]:
team_city_dict = {'Atlanta Hawks': 'Atlanta city',
    'Boston Celtics': 'Boston city',
    'Brooklyn Nets': 'Brooklyn city',
    'Charlotte Hornets': 'Charlotte city',
    'Chicago Bulls': 'Chicago city',
    'Cleveland Cavaliers': 'Cleveland city',
    'Dallas Mavericks': 'Dallas city',
    'Denver Nuggets': 'Denver city',
    'Detroit Pistons': 'Detroit city',
    'Golden State Warriors': 'San Francisco city',
    'Houston Rockets': 'Houston city',
    'Indiana Pacers': 'Indianapolis city',
    'Los Angeles Clippers': 'Los Angeles city',
    'Los Angeles Lakers': 'Los Angeles city',
    'Memphis Grizzlies': 'Memphis city',
    'Miami Heat': 'Miami city',
    'Milwaukee Bucks': 'Milwaukee city',
    'Minnesota Timberwolves': 'Minneapolis city',
    'New Orleans Pelicans': 'New Orleans city',
    'New York Knicks': 'New York city',
    'Oklahoma City Thunder': 'Oklahoma city',
    'Orlando Magic': 'Orlando city',
    'Philadelphia 76ers': 'Philadelphia city',
    'Phoenix Suns': 'Phoenix city',
    'Portland Trail Blazers': 'Portland city',
    'Sacramento Kings': 'Sacramento city',
    'San Antonio Spurs': 'San Antonio city',
    'Toronto Raptors': 'Toronto city',
    'Utah Jazz': 'Salt Lake city',
    'Washington Wizards': 'Washington city'}

NBA_revenue['City'] = NBA_revenue['Team'].map(team_city_dict)

In [454]:
NBA_revenue

,Year,Rank,Team,GateRevenue,OtherRevenue,TotalRevenues,PlayerExpenses,OtherExpenses,TotalExpenses,Operatingincome,Year2,City
0,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,2000,Los Angeles city
1,2000-2001,NaN,New York Knicks,NaN,NaN,157.0,81.6,46.9,NaN,28.5,2000,New York city
2,2000-2001,NaN,Chicago Bulls,NaN,NaN,117.0,32.7,32.6,NaN,51.7,2000,Chicago city
3,2000-2001,NaN,Portland Trail Blazers,NaN,NaN,101.0,89.5,30.8,NaN,-19.3,2000,Portland city
4,2000-2001,NaN,Philadelphia 76ers,NaN,NaN,103.0,53.6,38.3,NaN,11.1,2000,Philadelphia city
...,...,...,...,...,...,...,...,...,...,...,...,...
622,2020-2021,26.0,Indiana Pacers,32.0,232.0,264.0,127.0,72.0,199.0,65.0,2020,Indianapolis city
623,2020-2021,27.0,Charlotte Hornets,31.0,238.0,269.0,105.0,70.0,175.0,94.0,2020,Charlotte city
624,2020-2021,28.0,Minnesota Timberwolves,44.0,227.0,271.0,134.0,50.0,184.0,87.0,2020,Minneapolis city
625,2020-2021,29.0,Memphis Grizzlies,44.0,228.0,272.0,93.0,72.0,165.0,107.0,2020,Memphis city


In [455]:
City_pop_00 = pd.read_csv(path + 'Population00-10.csv')
City_pop_10 = pd.read_csv(path + 'Population10-20.csv')

In [456]:
merged_df = pd.merge(City_pop_00, City_pop_10, on='NAME', how='outer')

In [457]:

merged_df = merged_df.rename(columns={'NAME': 'City',
                                      'POPESTIMATE2000': '2000',
                                      'POPESTIMATE2001': '2001',
                                      'POPESTIMATE2002': '2002',
                                      'POPESTIMATE2003': '2003',
                                      'POPESTIMATE2004': '2004',
                                      'POPESTIMATE2005': '2005',
                                      'POPESTIMATE2006': '2006',
                                      'POPESTIMATE2007': '2007',
                                      'POPESTIMATE2008': '2008',
                                      'POPESTIMATE2009': '2009',
                                      'POPESTIMATE2010': '2010',
                                      'POPESTIMATE2011': '2011',
                                      'POPESTIMATE2012': '2012',
                                      'POPESTIMATE2013': '2013',
                                      'POPESTIMATE2014': '2014',
                                      'POPESTIMATE2015': '2015',
                                      'POPESTIMATE2016': '2016',
                                      'POPESTIMATE2017': '2017',
                                      'POPESTIMATE2018': '2018',
                                      'POPESTIMATE2019': '2019',
                                      'POPESTIMATE2020': '2020'})

merged_df = merged_df[['City', '2000', '2001', '2002', '2003', '2004', '2005',
                       '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                       '2013', '2014', '2015', '2016', '2017', '2018', '2019',
                       '2020']]


merged_df = merged_df.melt(id_vars=['City'], var_name='Year2', value_name='CityPopulation')


In [458]:
result_df = pd.merge(NBA_revenue, merged_df, on=['City', 'Year2'], how='left')

In [459]:
result_df

,Year,Rank,Team,GateRevenue,OtherRevenue,TotalRevenues,PlayerExpenses,OtherExpenses,TotalExpenses,Operatingincome,Year2,City,CityPopulation
0,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,2000,Los Angeles city,3701062.0
1,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,2000,Los Angeles city,3701062.0
2,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,2000,Los Angeles city,3701062.0
3,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,2000,Los Angeles city,3701062.0
4,2000-2001,NaN,New York Knicks,NaN,NaN,157.0,81.6,46.9,NaN,28.5,2000,New York city,8017608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54283,2020-2021,29.0,Memphis Grizzlies,44.0,228.0,272.0,93.0,72.0,165.0,107.0,2020,Memphis city,1996.0
54284,2020-2021,30.0,New Orleans Pelicans,47.0,221.0,268.0,121.0,53.0,174.0,94.0,2020,New Orleans city,389476.0
54285,2020-2021,30.0,New Orleans Pelicans,47.0,221.0,268.0,121.0,53.0,174.0,94.0,2020,New Orleans city,389476.0
54286,2020-2021,30.0,New Orleans Pelicans,47.0,221.0,268.0,121.0,53.0,174.0,94.0,2020,New Orleans city,389476.0


In [460]:
result_df = result_df.sort_values('CityPopulation', ascending=False).groupby(['Year', 'Team']).first().reset_index()

In [461]:
NBA_attend['Team'] = NBA_attend['Team'].str.title()

In [462]:
NBA_attend_melt = NBA_attend.melt(id_vars=['Team'], var_name='Year', value_name='Attendance')

In [463]:
NBA_attend_melt['Year'] = NBA_attend_melt['Year'].str[:4] + '-' + NBA_attend_melt['Year'].str[:2] + NBA_attend_melt['Year'].str[-2:]


In [464]:
NBA_attend_melt

,Team,Year,Attendance
0,Anderson Packers,2000-2001,NaN
1,Atlanta Hawks,2000-2001,560324.0
2,Baltimore Bullets,2000-2001,NaN
3,Boston Celtics,2000-2001,629201.0
4,Broolyn Nets,2000-2001,NaN
...,...,...,...
926,Tri-Cities Blackhawks,2018-2019,NaN
927,Utah Jazz,2018-2019,750546.0
928,Vancouver Grizzlies,2018-2019,NaN
929,Washington Capitals- Bullets/Wizards,2018-2019,716996.0


In [465]:
pop_att_merged = result_df.merge(NBA_attend_melt, on=['Team', 'Year'], how='left')

In [466]:
state_dict = {'Atlanta city': 'Georgia',
              'Boston city': 'Massachusetts',
              'Brooklyn city': 'New York',
              'Charlotte city': 'North Carolina',
              'Chicago city': 'Illinois',
              'Cleveland city': 'Ohio',
              'Dallas city': 'Texas',
              'Denver city': 'Colorado',
              'Detroit city': 'Michigan',
              'Houston city': 'Texas',
              'Indianapolis city': 'Indiana',
              'Los Angeles city': 'California',
              'Memphis city': 'Tennessee',
              'Miami city': 'Florida',
              'Milwaukee city': 'Wisconsin',
              'Minneapolis city': 'Minnesota',
              'New Orleans city': 'Louisiana',
              'New York city': 'New York',
              'Oklahoma city': 'Oklahoma',
              'Orlando city': 'Florida',
              'Philadelphia city': 'Pennsylvania',
              'Phoenix city': 'Arizona',
              'Portland city': 'Oregon',
              'Sacramento city': 'California',
              'Salt Lake city': 'Utah',
              'San Antonio city': 'Texas',
              'San Francisco city': 'California',
              'Toronto city': 'Ontario',
              'Washington city': 'District of Columbia'}

# Adding a new column called "State" to the "pop_att_merged" dataframe
pop_att_merged['State'] = pop_att_merged['City'].map(state_dict)

In [467]:
Median_Income = pd.read_excel(path + 'h08.xlsx')

In [468]:
Median_Income = Median_Income.melt(id_vars=['State'], var_name='Year2', value_name='StateMedianIncome')

In [469]:
Median_Income

,State,Year2,StateMedianIncome
0,United States,2021,70784
1,Alabama,2021,56929
2,Alaska,2021,81133
3,Arizona,2021,70821
4,Arkansas,2021,50784
...,...,...,...
1191,Virginia,2000,47163
1192,Washington,2000,42525
1193,West Virginia,2000,29411
1194,Wisconsin,2000,45088


In [470]:
Median_Income['State'] = Median_Income['State'].astype(str)
Median_Income['StateMedianIncome'] = Median_Income['StateMedianIncome'].astype(float)
Median_Income['Year2'] = Median_Income['Year2'].astype(str)

In [471]:
pop_att_merged['State'] = pop_att_merged['State'].astype(str)
pop_att_merged['Year2'] = pop_att_merged['Year2'].astype(str)

In [475]:
inc_pop_att_merged = pd.merge(pop_att_merged, Median_Income, on=['State', 'Year2'], how='left')

In [477]:
inc_pop_att_merged = inc_pop_att_merged.rename(columns={'StateMedianIncome': 'StateMedianIncome(2021$)'})

In [480]:
inc_pop_att_merged

,Year,Team,Rank,GateRevenue,OtherRevenue,TotalRevenues,PlayerExpenses,OtherExpenses,TotalExpenses,Operatingincome,Year2,City,CityPopulation,Attendance,State,StateMedianIncome(2021$)
0,2000-2001,Atlanta Hawks,NaN,NaN,NaN,76.0,42.2,28.0,NaN,5.8,2000,Atlanta city,417534.0,560324.0,Georgia,41901.0
1,2000-2001,Boston Celtics,NaN,NaN,NaN,81.0,49.5,21.7,NaN,9.8,2000,Boston city,591844.0,629201.0,Massachusetts,46753.0
2,2000-2001,Brooklyn Nets,NaN,NaN,NaN,73.0,59.7,24.6,NaN,-11.3,2000,Brooklyn city,11513.0,NaN,New York,40744.0
3,2000-2001,Charlotte Hornets,NaN,NaN,NaN,65.0,49.4,16.1,NaN,-0.5,2000,Charlotte city,577676.0,NaN,North Carolina,38317.0
4,2000-2001,Chicago Bulls,NaN,NaN,NaN,117.0,32.7,32.6,NaN,51.7,2000,Chicago city,2891582.0,NaN,Illinois,46064.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,2020-2021,Sacramento Kings,18.0,60.0,219.0,279.0,141.0,109.0,250.0,29.0,2020,Sacramento city,512838.0,NaN,California,77652.0
623,2020-2021,San Antonio Spurs,20.0,69.0,237.0,306.0,118.0,89.0,207.0,99.0,2020,San Antonio city,1567118.0,NaN,Texas,68404.0
624,2020-2021,Toronto Raptors,11.0,55.0,244.0,299.0,137.0,76.0,213.0,86.0,2020,Toronto city,4902.0,NaN,Ontario,NaN
625,2020-2021,Utah Jazz,19.0,68.0,240.0,308.0,151.0,99.0,250.0,58.0,2020,Salt Lake city,NaN,NaN,Utah,83993.0
